In [9]:
import pandas as pd
import transformers
import torch
conv_data = pd.read_csv('dataset/100_conversation.csv')

In [10]:
conv_data.head()

,conversation,conv_id
0,Are you a fan of Google or Microsoft?\nBoth ar...,1
1,do you like dance?\nYes I do. Did you know Br...,2
2,Hey what's up do use Google very often?I reall...,3
3,Hi! do you like to dance?\nI love to dance a ...,4
4,do you like dance?\nI love it. Did you know Br...,5


In [11]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto" # if you have GPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [43]:
from torch.utils.data import Dataset
from transformers.pipelines.pt_utils import KeyDataset
# Make the prompt into the standard format
class summary_dataset(Dataset):
    
    def __init__(self,data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return f'For the convertation summarize in a paragraph it such that all the\
        important points are covered the conversation is as follows ```{self.data.conversation[idx]}``` \n Summary:'
dataset = summary_dataset(conv_data)

In [54]:
dataset[1]

'For the convertation summarize in a paragraph it such that all the        important points are covered the conversation is as follows ```do you like dance?\nYes  I do. Did you know Bruce Lee was a cha cha dancer?\nYes he even won a hardcore cha cha championship in 1958\nYeah. Did you know Tupac was a ballet dancer?\nYes and he even was in the production of the nutcracker\nYeah. Ballet dancer go through 4 pairs of shoes a week\nYes that is a lot of shoes and also a lot of money\nYeah true. Did you know babies are really good at dancing?\nYes and they smile more when they hit the beat\nYeah they are much smarter than we give them credit for\nTrue Did you know Jackson had a patent on a dancing device?\nYes it helped him smooth out his dance moves\nNice. Do you like Shakespeare?\nYes I do. Do you know that he popularized many phrases\n Yes like good riddance, in my heart of hearts and such\n Yes and then he also invented names like Jessica, Olivia and Miranda\nYes. And for his works you h

In [35]:
from tqdm import tqdm

In [57]:
import ipdb

In [60]:
coversations,summarizations = [],[]
for idx in tqdm(range(len(dataset)),desc="Generation: "):
    # print(idx)
    out = dataset[idx]
    
    sequences = pipeline(out,
                        do_sample=True,
                        top_k=10,
                        top_p = 0.9,
                        temperature = 0.2,
                        num_return_sequences=1,
                        eos_token_id=tokenizer.eos_token_id,
                        max_length=2048, # can increase the length of sequence 
                        )
    # ipdb.set_trace()
    conv,summ = sequences[0]['generated_text'].split("\n Summary:")
    # print(conv,summ)
    coversations.append(conv)
    summarizations.append(summ)

Generation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:10<00:00,  6.11s/it]


In [61]:
summ = pd.DataFrame({"conversations":coversations,"summarizations":summarizations})
summ.to_csv("dataset/summaries.csv",index=False)

In [62]:
summ.head()

,conversations,summarizations
0,For the convertation summarize in a paragraph ...,The conversation is between two people who a...
1,For the convertation summarize in a paragraph ...,\nThe conversation revolves around various to...
2,For the convertation summarize in a paragraph ...,"The conversation revolves around Google, a t..."
3,For the convertation summarize in a paragraph ...,The conversation is about dancing and differ...
4,For the convertation summarize in a paragraph ...,"\nThe conversation revolves around dance, Bru..."


In [1]:
import pandas as pd

In [2]:
tab = pd.read_csv("dataset/summaries.csv")

In [3]:
tab

,conversations,summarizations
0,For the convertation summarize in a paragraph ...,The conversation is between two people who a...
1,For the convertation summarize in a paragraph ...,\nThe conversation revolves around various to...
2,For the convertation summarize in a paragraph ...,"The conversation revolves around Google, a t..."
3,For the convertation summarize in a paragraph ...,The conversation is about dancing and differ...
4,For the convertation summarize in a paragraph ...,"\nThe conversation revolves around dance, Bru..."
...,...,...
95,For the convertation summarize in a paragraph ...,"The conversation is about pets, specifically..."
96,For the convertation summarize in a paragraph ...,\nThe conversation covers a range of topics i...
97,For the convertation summarize in a paragraph ...,"\nThe conversation revolves around cooking, f..."
98,For the convertation summarize in a paragraph ...,The conversation covers a variety of topics ...


In [4]:
# Metrics Bert Score
# NER and check its overlap (IOU)
# Rouge and Bleu Meteor Chrf++
# Hugging faces evaluate

In [5]:
# Hugging faces evaluate

In [6]:
from evaluate import load
bertscore = load("bertscore")

In [7]:
from tqdm import tqdm
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
def cal():
    precision,recall,f1 = 0,0,0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = bertscore.compute(predictions=[b['summarizations']], references=[b['conversations']], lang="en")
        # print(out)
        precision += out['precision'][0]
        recall += out['recall'][0]
        f1 += out['f1'][0]
        
    print(f"The bertscore based scores \n Precision: {precision/len(tab)} \n Recall: {recall/len(tab)} \n F1: {f1/len(tab)}")

In [30]:
cal()

100it [00:06, 15.58it/s]

The bertscore based scores 
 Precision: 0.8656682378053665 
 Recall: 0.7922304481267929 
 F1: 0.8272823590040207


In [34]:
bleu_score = load('google_bleu')

In [ ]:
def cal():
    bleu_s = 0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = bleu_score.compute(predictions=[b['summarizations']], references=[b['conversations']])
        # print(out)
        bleu_s += out['google_bleu']
        
    print(f"The BLEU score: {bleu_s/len(tab)}")

In [ ]:
cal()

100it [00:01, 54.00it/s]

The BLEU score: 0.059690493741630685


In [43]:
chrf = load("chrf")
def cal():
    bleu_s = 0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = chrf.compute(predictions=[b['summarizations']], references=[b['conversations']])
        # print(out)
        bleu_s += out['score']
        
    print(f"The chrf score: {bleu_s/len(tab)}")
cal()

100it [00:01, 94.99it/s]

The chrf score: 18.893789145698715


In [8]:
rouge = load('rouge')

In [17]:
def cal():
    precision,recall,f1 = 0,0,0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = rouge.compute(predictions=[b['summarizations']], references=[b['conversations']])
        # print(out)
        precision += out['rouge1']
        # recall += out['rouge2']
        f1 += out['rougeL']
        
    print(f"The Rouge based scores \n rouge1: {precision/len(tab)} \n rougeL: {f1/len(tab)}")

In [18]:
cal()

100it [00:21,  4.76it/s]


The Rouge based scores 
 rouge1: 0.24814070160621415 
 rougeL: 0.16468335879954318


## Future work 
1. Generation from Mistra/GPT-4 and fine tune on llama2